In [52]:
import requests
import pandas as pd

In [119]:
ids_df.country.unique()

array(['Portugal', 'Spain', 'France', 'Germany', 'United Kingdom',
       'Italy', 'Austria', 'Netherlands', 'Switzerland', 'Greece'],
      dtype=object)

In [236]:
ids_df = pd.read_parquet("../data/touristic_spots.parquet")
city = "Lyon"
country = "France"
url = "http://127.0.0.1:8080/trace_route"
place_ids = ids_df.loc[(ids_df.city == city) & (ids_df.country == country)].id.to_list()
j = {
    "city":city,
    "country":country,
    "place_ids":place_ids,
    "nr_of_routes":3,
    "max_total_time":6
}
place_ids = [str(i) for i in place_ids]


In [237]:
%%time
rs = requests.post(url,json=j)

CPU times: user 6.68 ms, sys: 6.82 ms, total: 13.5 ms
Wall time: 4.11 s


In [238]:
rs.json()

[{'distance': 10119,
  'route_indexes': [0, 7, 15, 6, 14, 8, 10, 2, 5, 13, 1, 12, 4, 11, 3, 9, 0],
  'route_names': ['1000362',
   '1000370',
   '1000380',
   '1000368',
   '1000379',
   '1000372',
   '1000374',
   '1000364',
   '1000367',
   '1000378',
   '1000363',
   '1000376',
   '1000366',
   '1000375',
   '1000365',
   '1000373',
   '1000362']},
 {'distance': 0,
  'route_indexes': [0, 0],
  'route_names': ['1000369', '1000369']},
 {'distance': 1364,
  'route_indexes': [0, 1, 2, 0],
  'route_names': ['1000361', '1000371', '1000377', '1000361']}]

In [99]:
distance_matrix = pd.read_parquet(f"../data/distance_matrices/walking/{country}/{city}.parquet")


In [242]:
for i in rs.json()[2]["route_names"]:
    print(ids_df.loc[ids_df.id == int(i)].place_names)

360    Parc de la Tête d'Or
Name: place_names, dtype: object
370    Lyon Zoo
Name: place_names, dtype: object
376    Le Petit Train du Parc de la Tête d'Or
Name: place_names, dtype: object
360    Parc de la Tête d'Or
Name: place_names, dtype: object


In [22]:
Belém Tower

['1000001',
 '1000010',
 '1000009',
 '1000008',
 '1000007',
 '1000006',
 '1000005',
 '1000004',
 '1000003',
 '1000002',
 '1000001']

In [133]:
a = pd.read_excel("/Users/rennanaraujo/Neobrain_Git/market-analysis/data/activity_sector_updated.xlsx")
activity_sectors = dict(
            zip(a.secteur_activite_libelle, a.updated_sector)
        )

In [135]:
a.updated_sector.unique()

array(['Automotive / Aerospace / Transport', 'Services',
       'Retail /Luxury / Consumer Goods', 'Other', 'Industry',
       'Technologies / Média / Télécommunications', 'Health Care',
       'Banking / Finance / Insurance', 'Public Sector', 'Travel'],
      dtype=object)

In [104]:
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
import requests
import pandas as pd

ids_df = pd.read_parquet("../data/touristic_spots.parquet")
city = "Porto"
country = "Portugal"
url = "http://127.0.0.1:8080/trace_route"
place_ids = ids_df.loc[(ids_df.city == city) & (ids_df.country == country)][:10].id.to_list()
j = {
    "city":city,
    "country":country,
    "place_ids":place_ids
}
place_ids = [str(i) for i in place_ids]

class TSPSolver:
    def __init__(self, distance_matrix,places):
        self.distance_matrix = distance_matrix
        self.num_vehicles = 1
        self.depot = 0
        self.places = places

    def solve(self,max_time=3):
        """Solve the TSP problem and return the solution."""
        # Create the routing index manager.
        manager = pywrapcp.RoutingIndexManager(len(self.distance_matrix),
                                               self.num_vehicles, self.depot)

        # Create Routing Model.
        routing = pywrapcp.RoutingModel(manager)

        # Create and register a transit callback.
        def distance_callback(from_index, to_index):
            """Returns the distance between the two nodes."""
            from_node = manager.IndexToNode(from_index)
            to_node = manager.IndexToNode(to_index)
            return self.distance_matrix[from_node][to_node]

        transit_callback_index = routing.RegisterTransitCallback(distance_callback)

        # Define cost of each arc.
        routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

        # Set the solver parameters.
        search_parameters = pywrapcp.DefaultRoutingSearchParameters()
        search_parameters.local_search_metaheuristic = (
            routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)
        search_parameters.time_limit.seconds = max_time

        # Solve the problem.
        solution = routing.SolveWithParameters(search_parameters)

        # Return the solution.
        if solution:
            return self._get_solution(manager, routing, solution,self.places)

    @staticmethod
    def _get_solution(manager, routing, solution,places):
        """Return the solution as a dictionary."""
        index = routing.Start(0)
        route = [manager.IndexToNode(index)]
        distance = 0
        while not routing.IsEnd(index):
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route.append(manager.IndexToNode(index))
            distance += routing.GetArcCostForVehicle(previous_index, index, 0)
        return {
            "route_names": [places[i] for i in route],
            "route_indexes":route,
            "distance": distance
        }
    
solver = TSPSolver(distance_matrix.loc[place_ids, place_ids],place_ids)
rs =solver.solve(max_time=10)

In [132]:
len(a.updated_sector.unique())

10

In [105]:
solver = TSPSolver(distance_matrix.loc[place_ids, place_ids],place_ids)

In [106]:
rs =solver.solve(max_time=10)

/Users/rennanaraujo/opt/anaconda3/lib/python3.9/site-packages/ortools/constraint_solver/pywrapcp.py:4582: RuntimeWarning: Unexpected end-group tag: Not all data was converted
  return _pywrapcp.DefaultRoutingSearchParameters()
E0328 08:50:32.864756     1 routing.cc:2605] Invalid RoutingSearchParameters: local_search_neighborhood_operator.use_relocate should be set to BOOL_TRUE or BOOL_FALSE instead of BOOL_UNSPECIFIED (value: 0)


In [107]:
rs

In [91]:
distance_matrix

,1000021,1000022,1000023,1000024,1000025,1000026,1000027,1000028,1000029,1000030,1000031,1000032,1000033,1000034,1000035,1000036,1000037,1000038,1000039,1000040
1000021,0,562,653,780,1474,5686,4841,639,2590,7033,771,959,268,698,1391,165,2537,558,864,799
1000022,563,0,588,282,1770,6123,5278,935,2703,7470,460,461,630,1009,898,592,2974,17,999,1235
1000023,653,730,0,841,1221,5678,4833,612,2207,7025,328,1020,384,881,1290,774,2529,680,354,988
1000024,780,282,841,0,1987,6340,5495,1152,2834,7687,591,399,847,1226,1009,809,3191,265,1130,1452
1000025,1451,1746,1220,2060,0,4723,3665,1186,1128,6070,1438,2239,1245,1005,2509,1602,1574,1899,896,1000
1000026,5686,6171,5678,6389,4953,0,2722,5254,5026,2091,6005,6568,5702,5056,7000,5691,3149,6167,5455,4887
1000027,4841,5326,4833,5544,3665,2722,0,4409,3260,3144,5028,5723,4857,4211,6068,4846,2304,5322,4635,4042
1000028,615,910,612,1128,1185,5257,4412,0,2301,6604,939,1307,409,303,1739,895,2108,906,752,410
1000029,2568,2724,2208,2835,1129,5015,3261,2303,0,5201,2244,3014,2362,2122,3394,2719,1955,2674,1851,2117
1000030,7033,7518,7025,7736,6092,2091,3145,6601,5201,0,7352,7915,7049,6403,8347,7038,4496,7514,6802,6234
